In [2]:
from darts.models import TFTModel
# from darts import GridSearch
from darts.metrics import mape
import numpy as np
import pandas as pd

ImportError: cannot import name 'GridSearch' from 'darts' (C:\Users\siljo\AppData\Local\Programs\Python\Python311\Lib\site-packages\darts\__init__.py)

In [ ]:
file_path = 'processed_datasetNew.csv'
data = pd.read_csv(file_path)

data = data.set_index('Date')
data.index = pd.to_datetime(data.index)

In [ ]:
split_date = pd.to_datetime('2008-12-31 23:59')
data = data[data.index >= split_date]

In [ ]:
data["Temperature t+24"] = data["Temperature"].shift(periods=-24)
data["Humidity t+24"] = data["Humidity"].shift(periods=-24)
data["Month"] = data.index.month
data["Month_sin"] = np.sin(2 * np.pi * data.index.month / 12)
data["Month_cos"] = np.cos(2 * np.pi * data.index.month / 12)
data["Month t+24"] = data["Month"].shift(periods=-24)
data["Month_sin t+24"] = data["Month_sin"].shift(periods=-24)
data["Month_cos t+24"] = data["Month_cos"].shift(periods=-24)
data["Quarter"] = data.index.quarter
data["Quarter_sin"] = np.sin(2 * np.pi * data.index.quarter / 4)
data["Quarter_cos"] = np.cos(2 * np.pi * data.index.quarter / 4)
data["Quarter t+24"] = data["Quarter"].shift(periods=-24)
data["Quarter_sin t+24"] = data["Quarter_sin"].shift(periods=-24)
data["Quarter_cos t+24"] = data["Quarter_cos"].shift(periods=-24)
data["Year"] = data.index.year
data["Year t+24"] = data["Year"].shift(periods=-24)
data['week_cos'] =  np.cos(2 * np.pi * data.index.isocalendar().week / 53)
data['week_sin'] =  np.sin(2 * np.pi * data.index.isocalendar().week / 53)
data["week_sin t+24"] = data["week_sin"].shift(periods=-24)
data["week_cos t+24"] = data["week_cos"].shift(periods=-24)
data['weekday_cos'] = np.sin(2 * np.pi * (data.index.weekday+1) / 7)
data['weekday_sin'] = np.cos(2 * np.pi * (data.index.weekday+1) / 7)
data["weekday_sin t+24"] = data["weekday_sin"].shift(periods=-24)
data["weekday_cos t+24"] = data["weekday_cos"].shift(periods=-24)

In [ ]:
features_options = [
    ["Temperature", "Humidity"],
    ["Temperature", "Humidity", "Year", "Month"],
    ["Temperature", "Humidity", "Year", "Month", "Month_cos", "Month_sin"],
    ["Temperature", "Humidity", "Year", "Month", "Month_cos", "Month_sin", "weekday_cos", "weekday_sin"],
    ["Temperature", "Humidity", "Temperature t+24", "Humidity t+24"],
    ["Temperature", "Humidity", "Year", "Month", "Temperature t+24", "Humidity t+24", "Year t+24", "Month t+24"],
    ["Temperature", "Humidity", "Year", "Month", "Month_cos", "Month_sin", "Temperature t+24", "Humidity t+24", "Year t+24", 
    "Month t+24", "Month_cos t+24", "Month_sin t+24"],
    ["Temperature", "Humidity", "Year", "Month", "Month_cos", "Month_sin", "weekday_cos", "weekday_cos", "Temperature t+24", "Humidity t+24",
    "Year t+24", "Month t+24", "Month_cos t+24", "Month_sin t+24", "weekday_cos t+24", "weekday_sin t+24"],
    ["Temperature", "Humidity", "Year", "weekday_cos", "weekday_sin", "Month_cos", "Month_sin"],
    ["Temperature", "Humidity", "Year", "weekday_cos", "weekday_sin", "Temperature t+24", "Humidity t+24", "Year t+24", "weekday_cos t+24", "weekday_sin t+24"]
]

In [ ]:
data_sin = data[['SIN']]
data_cov = data[features_options[8]]

In [ ]:
data_sin = data[['SIN']]
data_cov = data[features_options[8]]

In [ ]:
covariates = covariates.add_holidays('PY')

In [ ]:
# # Create training and validation sets:
training_cutoff = pd.Timestamp("20191231T230000")
train, val = series.split_after(training_cutoff)
# validation_cutoff = pd.Timestamp("20211231T230000")
# val, test = val.split_after(validation_cutoff)

In [ ]:
# Normalize the time series (note: we avoid fitting the transformer on the validation set)
transformer = Scaler()
train_transformed = transformer.fit_transform(train)
val_transformed = transformer.transform(val)
series_transformed = transformer.transform(series)

In [ ]:
covariates = covariates.stack(
    TimeSeries.from_times_and_values(
        times=series.time_index,
        values=np.arange(len(series)),
        columns=["linear_increase"],
    )
)
covariates = covariates.astype(np.float32)
cov_train, cov_val = covariates.split_after(training_cutoff)
# cov_val, cov_test = covariates.split_after(validation_cutoff)

In [ ]:
# transform covariates (note: we fit the transformer on train split and can then transform the entire covariates series)
scaler_covs = Scaler()
covariates_transformed_train = scaler_covs.fit_transform(cov_train)
covariates_transformed_val = scaler_covs.transform(cov_val)
covariates_transformed = scaler_covs.transform(covariates)

In [ ]:
import time
st = time.time()

In [ ]:
parameter_grid = {
    'input_chunk_length': [300, 680],
    'output_chunk_length': [24, 24],
    'hidden_size': [35, 64],
    'dropout_rate': [0.08, 0.2],  
    'lstm_layers': [1, 4],
    'batch_size': [15, 25]
}

In [ ]:
best_model, best_params, best_metric = gridsearch(
    parameters=parameter_grid,
    model_class=TFTModel,
    series=train_transformed,  # The TimeSeries object used for training
    val_series=val_transformed,  # The TimeSeries object used for validation
    # past_covariates=past_covariates,  # Optionally, your past covariates
    future_covariates=covariates_transformed,  # Optionally, your future covariates
    metric=mape,  # The metric to optimize for
    verbose=True,  # If you want to see progress
    n_jobs=-1  # Use all available cores for parallel processing
)

In [ ]:
et = time.time()
elapsed_time = et - st
print(elapsed_time, 'seconds.')
print(elapsed_time/60, 'minutes.')
print(elapsed_time/(60*60), 'hours.')

In [ ]:
print(f"Best parameters: {best_params}")
print(f"Best MAPE: {best_metric}")